# 3단계: ML 파이프라인 추가

<div class="alert alert-warning"> 이 노트북은 <code>SageMaker Distribution Image 3.6.1</code>을 사용하는 SageMaker Studio JupyterLab 인스턴스에서 SageMaker Python SDK 버전 <code>2.255.0</code>으로 마지막으로 테스트되었습니다</div>

이 단계에서는 [Amazon SageMaker Pipelines](https://aws.amazon.com/sagemaker/pipelines/)와 [Amazon SageMaker Model Registry](https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry.html)를 사용하여 엔드투엔드 ML 워크플로우를 자동화합니다. [Amazon SageMaker Feature Store](https://aws.amazon.com/sagemaker/feature-store/)를 사용하여 피처 엔지니어링을 재사용 가능하고, 반복 가능하며, 확장 가능하게 만듭니다.

||||
|---|---|---|
|1. |노트북에서 실험 ||
|2. |SageMaker AI 처리 작업 및 SageMaker SDK로 확장 ||
|3. |ML 파이프라인, 모델 레지스트리 및 피처 스토어로 운영화 |**<<<< 현재 위치**|
|4. |모델 구축 CI/CD 파이프라인 추가 ||
|5. |모델 배포 파이프라인 추가 ||
|6. |모델 및 데이터 모니터링 추가 ||


<div class="alert alert-info"> 이 노트북에서는 JupyterLab에서 <code>Python 3</code> 커널을 사용하고 있는지 확인하세요.</div>

In [ ]:
# To get the Feature Processor module, we need to reinstall the SageMaker python SDK along with extra dependencies
%pip install 'sagemaker[feature-processor]<3' --force-reinstall

In [ ]:
%pip install --force-reinstall --no-cache s3fs boto3

In [ ]:
# %pip install --force-reinstall --no-cache boto3

In [ ]:
# Restart kernel
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
import pandas as pd
import json
import boto3
import pathlib
import io
import sagemaker
import mlflow
from time import gmtime, strftime, sleep
from sagemaker.deserializers import CSVDeserializer
from sagemaker.serializers import CSVSerializer
from importlib.metadata import version, PackageNotFoundError

from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import (
    ProcessingInput, 
    ProcessingOutput, 
    ScriptProcessor
)
from sagemaker.inputs import TrainingInput

from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import (
    ProcessingStep, 
    TrainingStep, 
    CreateModelStep,
    CacheConfig
)
from sagemaker.workflow.check_job_config import CheckJobConfig
from sagemaker.workflow.parameters import (
    ParameterInteger, 
    ParameterFloat, 
    ParameterString, 
    ParameterBoolean
)
from sagemaker.workflow.quality_check_step import (
    DataQualityCheckConfig,
    ModelQualityCheckConfig,
    QualityCheckStep,
)
from sagemaker.workflow.clarify_check_step import (
    ModelBiasCheckConfig, 
    ClarifyCheckStep, 
    ModelExplainabilityCheckConfig
)
from sagemaker import Model
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.conditions import (
    ConditionGreaterThan,
    ConditionGreaterThanOrEqualTo
)
from sagemaker.workflow.parallelism_config import ParallelismConfiguration
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import (
    Join,
    JsonGet
)
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)
from sagemaker.lambda_helper import Lambda

from sagemaker.model_metrics import (
    MetricsSource, 
    ModelMetrics, 
    FileSource
)
from sagemaker.drift_check_baselines import DriftCheckBaselines
from sagemaker.workflow.pipeline_definition_config import PipelineDefinitionConfig 
from sagemaker.image_uris import retrieve
from sagemaker.workflow.function_step import step
from sagemaker.workflow.step_outputs import get_step
from sagemaker.model_monitor import DatasetFormat, model_monitoring
from IPython.display import HTML

(sagemaker.__version__, boto3.__version__, mlflow.__version__)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%store -r 

%store

try:
    initialized
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN 00-start-here notebook   ")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")

## Constants 설정

In [ ]:
# Set names of pipeline objects, experiment, and a model
project = "from-idea-to-prod"

current_timestamp = strftime('%d-%H-%M-%S', gmtime())

registered_model_name = f"{project}-pipeline-model-{current_timestamp}"
experiment_name = f"{project}-pipeline-{current_timestamp}"
pipeline_name = f"{project}-pipeline-{current_timestamp}"
pipeline_model_name = f"{project}-model-xgb"
model_package_group_name = registered_model_name
endpoint_config_name = f"{project}-endpoint-config"
endpoint_name = f"{project}-endpoint"
model_approval_status = "PendingManualApproval"

In [ ]:
# Set instance types and counts
process_instance_type = "ml.m5.large"
train_instance_type = "ml.m5.large"

In [ ]:
# Set S3 urls for various datasets produced in the pipeline
output_s3_prefix = f"s3://{bucket_name}/{bucket_prefix}"
output_s3_url = f"{output_s3_prefix}/output"

train_s3_url = f"{output_s3_prefix}/train"
validation_s3_url = f"{output_s3_prefix}/validation"
test_s3_url = f"{output_s3_prefix}/test"
evaluation_s3_url = f"{output_s3_prefix}/evaluation"

baseline_s3_url = f"{output_s3_prefix}/baseline"
baseline_results_s3_url = f"{baseline_s3_url}/results"

prediction_baseline_s3_url = f"{output_s3_prefix}/prediction_baseline"
prediction_baseline_results_s3_url=f"{prediction_baseline_s3_url}/results"

In [ ]:
XGBOOST_IMAGE_URI = sagemaker.image_uris.retrieve(
            "xgboost", 
            region=boto3.Session().region_name,
            version="1.7-1"
)

In [ ]:
%store train_s3_url
%store validation_s3_url
%store test_s3_url
%store baseline_s3_url
%store pipeline_name
%store model_package_group_name
%store evaluation_s3_url
%store prediction_baseline_s3_url
%store output_s3_url

In [ ]:
print(f"Train S3 url: {train_s3_url}")
print(f"Validation S3 url: {validation_s3_url}")
print(f"Test S3 url: {test_s3_url}")
print(f"Data baseline S3 url: {baseline_s3_url}")
print(f"Evaluation metrics S3 url: {evaluation_s3_url}")
print(f"Model prediction baseline S3 url: {prediction_baseline_s3_url}")

## 헬퍼 함수 정의
코드 가독성을 높이기 위한 단축 함수들을 정의합니다.

In [ ]:
def get_xgb_estimator(
    session,
    instance_type,
    output_s3_url,
    base_job_name,
):
    # Instantiate an XGBoost estimator object
    estimator = sagemaker.estimator.Estimator(
        image_uri=XGBOOST_IMAGE_URI,
        role=sagemaker.get_execution_role(), 
        instance_type=instance_type,
        instance_count=1,
        output_path=output_s3_url,
        sagemaker_session=session,
        base_job_name=base_job_name,
    )
    
    # Define algorithm hyperparameters
    estimator.set_hyperparameters(
        num_round=100, # the number of rounds to run the training
        max_depth=3, # maximum depth of a tree
        eta=0.5, # step size shrinkage used in updates to prevent overfitting
        alpha=2.5, # L1 regularization term on weights
        objective="binary:logistic",
        eval_metric="auc", # evaluation metrics for validation data
        subsample=0.8, # subsample ratio of the training instance
        colsample_bytree=0.8, # subsample ratio of columns when constructing each tree
        min_child_weight=3, # minimum sum of instance weight (hessian) needed in a child
        early_stopping_rounds=10, # the model trains until the validation score stops improving
        verbosity=1, # verbosity of printing messages
    )

    return estimator

## MLflow 구성

In [ ]:
# Check that the MLflow server is in the status 'Created' or 'Started'
sm = boto3.client("sagemaker")

while sm.describe_mlflow_tracking_server(TrackingServerName=mlflow_name)['TrackingServerStatus'] not in ['Created', 'Started']:
    print(f"The MLflow server {mlflow_name} is not in the status 'Created' or 'Started'")
    sleep(30)
else:
    print(f"Using server {mlflow_name}")

In [ ]:
mlflow.set_tracking_uri(mlflow_arn)
experiment = mlflow.set_experiment(experiment_name=experiment_name)

## AWS 인프라 기본값 구성
YAML 구성 파일을 사용하여 SageMaker API에 자동으로 전달되는 기본값(예: 작업 매개변수)을 정의할 수 있습니다. VPC ID, 보안 그룹, KMS 키 등과 같은 인프라 설정에 대한 정적 매개변수를 제공하거나 원격 함수와 작업할 때 특히 편리합니다.

예제와 자세한 내용은 [SageMaker Python SDK로 기본값 구성 및 사용](https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk) 문서를 참조하세요.

SageMaker 파이프라인은 이러한 `config.yaml` 파일을 사용합니다.

In [ ]:
# Print default location of configuration files
import os
from platformdirs import site_config_dir, user_config_dir

#Prints the location of the admin config file
print(os.path.join(site_config_dir("sagemaker"), "config.yaml"))

#Prints the location of the user config file
print(os.path.join(user_config_dir("sagemaker"), "config.yaml"))

다음 셀은 구성 파일을 생성하고 원격 함수에 대한 기본값을 설정합니다. 이러한 값은 `@step` 데코레이터에 자동으로 전달되므로 명시적으로 지정할 필요가 없습니다. 개발자 가이드의 [파이프라인 구성](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-step-decorator-cfg-pipeline.html)을 참조하세요.

In [ ]:
%%writefile config.yaml

SchemaVersion: '1.0'
SageMaker:
    PythonSDK:
        Modules:
            RemoteFunction:
                InstanceType: ml.m5.xlarge
                Dependencies: ./requirements.txt
                IncludeLocalWorkDir: true
                CustomFileFilter:
                    IgnoreNamePatterns: # files or directories to ignore
                        - "*.ipynb" # all notebook files
                        - "*.md" # all markdown files
                        - "__pycache__"

In [ ]:
# copy the configuration file to user config file location
%mkdir -p {user_config_dir("sagemaker")}
%cp config.yaml {os.path.join(user_config_dir("sagemaker"), "config.yaml")}

In [ ]:
# Alternatively instead of copying config.yaml to user config dir, you can point SageMaker to the configuration file
# os.environ["SAGEMAKER_USER_CONFIG_OVERRIDE"] = os.getcwd()

## 환경 준비

In [ ]:
# Install xgboost for the local testing of the code
%pip install -q xgboost

설치된 패키지의 버전을 가져와서 환경을 복제하기 위한 `requirements.txt` 파일을 생성합니다. SageMaker 파이프라인은 이를 사용하여 작업 컨테이너에서 환경을 설정합니다.

In [ ]:
if os.path.exists('requirements.txt'):
    os.remove('requirements.txt')
    print("Existing requirements.txt file deleted.")

In [ ]:
packages = ['xgboost', 'mlflow', 'sagemaker-mlflow','sagemaker']
requirements = [f'{p}=={version(p)}' for p in packages]

requirements.append('protobuf==3.20.1')
requirements.append('s3fs==0.4.2') #FIXME: find a more up to date version that works

if requirements:
    with open('requirements.txt', 'w') as f:
        f.write('\n'.join(requirements))
    print("\nNew requirements.txt file created with the following content:")
    print('\n'.join(requirements))
else:
    print("\nNo requirements.txt file created as no packages were found")

In [ ]:
# %%writefile requirements.txt
# scikit-learn
# pandas>=2.0.0
# s3fs==0.4.2
# sagemaker>=2.237
# xgboost
# mlflow==2.16.2
# sagemaker-mlflow==0.1.0

## SageMaker 파이프라인

### 파이프라인 매개변수 설정
SageMaker Pipelines는 [매개변수화](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-parameters.html)를 지원하여 파이프라인 코드를 변경하지 않고도 런타임에 입력 매개변수를 지정할 수 있습니다. [`sagemaker.workflow.parameters`](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#parameters) 모듈에서 사용 가능한 매개변수 클래스를 사용할 수 있습니다.
매개변수에는 기본값이 있으며, 파이프라인 실행을 시작할 때 매개변수 값을 지정하여 이를 재정의할 수 있습니다.

In [ ]:
# Set processing instance type
process_instance_type_param = ParameterString(
    name="ProcessingInstanceType",
    default_value=process_instance_type,
)

# Set training instance type
train_instance_type_param = ParameterString(
    name="TrainingInstanceType",
    default_value=train_instance_type,
)

# Set model approval status for the model registry
model_approval_status_param = ParameterString(
    name="ModelApprovalStatus",
    default_value=model_approval_status
)

# Minimal threshold for model performance on the test dataset
test_score_threshold_param = ParameterFloat(
    name="TestScoreThreshold",
    default_value=0.75
)

# Parametrize the S3 url for input dataset
input_s3_url_param = ParameterString(
    name="InputDataUrl",
    default_value=input_s3_url,
)

# Model package group name
model_package_group_name_param = ParameterString(
    name="ModelPackageGroupName",
    default_value=model_package_group_name,
)

# MLflow tracking server ARN
tracking_server_arn_param = ParameterString(
    name="TrackingServerARN",
    default_value=mlflow_arn,
)

### 파이프라인 단계 구현 및 테스트
다음과 같은 파이프라인을 생성합니다:
| 단계 | 설명 |
|---|---|
| **데이터 처리** | 피처 엔지니어링 및 데이터셋 분할을 위한 SageMaker 처리 작업 실행|
| **훈련** | XGBoost 알고리즘을 사용한 SageMaker 훈련 작업 실행 |
| **평가** | 훈련된 모델의 성능 평가 |
| **조건부 단계** | 모델의 성능이 지정된 임계값을 충족하는지 확인 |
| **모델 등록** | SageMaker 모델 레지스트리에 모델 버전 등록 |

파이프라인을 코드로 구현하는 것을 용이하게 하기 위해 SageMaker의 두 가지 유용한 기능인 서브클래스 호환성과 @step 데코레이터를 사용합니다.

#### 서브클래스 호환성
💡 [워크플로우 파이프라인 작업 단계](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#steps)에 서브클래스 호환성을 사용하여 작업 추상화를 구축하고 이전 단계 노트북의 처리, 훈련, 변환 및 튜닝 작업을 실행하는 코드와 정확히 동일한 코드를 사용하여 파이프라인을 구성할 수 있습니다. `sagemaker_session` 대신 [PipelineSession](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.pipeline_context.PipelineSession) 컨텍스트를 사용하여 `processor.run()` 또는 `estimator.fit()`과 같은 실행 호출을 캡처하되 파이프라인이 생성되고 실행될 때까지는 실행하지 않아야 합니다.

#### @step 데코레이터
💡 기존 Python 코드를 SageMaker 파이프라인으로 리프트 앤 시프트할 수 있습니다. 또한 Python 함수를 사용하여 SageMaker Python SDK로 ML 워크플로우를 구현하고 노트북에서 모든 코드를 로컬로 테스트할 수 있습니다. 파이프라인을 생성하려는 경우 SageMaker Python SDK [`@step`](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#step-decorator) 데코레이터를 사용하여 Python 함수를 파이프라인 단계로 변환할 수 있습니다. 자세한 내용과 예제는 SageMaker [개발자 가이드](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-step-decorator-create-pipeline.html)를 참조하세요.

다음 코드는 Python 함수를 사용하여 워크플로우 단계를 구현하고 로컬에서 테스트한 다음 `@step` 데코레이터를 적용하여 함수를 파이프라인 단계로 재사용합니다.

#### @step 데코레이터의 제한사항
파이프라인 단계에 `@step` 데코레이터를 사용할 때의 특정 [제한사항](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-step-decorator-limit.html)을 인지하세요.

다음 코드 셀에서 단계를 개발하고 테스트하기 시작합니다.

#### 처리 단계
2단계 [노트북](./02-sagemaker-containers.ipynb)의 Python 함수 로컬 코드를 재사용하여 `./pipeline_steps` 폴더에 `preprocess` 함수를 생성합니다. 이 함수를 사용하여 파이프라인 처리 단계를 생성합니다.

In [ ]:
# Python function code is in the local files
from pipeline_steps.preprocess import preprocess

In [ ]:
# See the function code
# !pygmentize pipeline_steps/preprocess.py

In [ ]:
# check that there is dataset under S3 url
!aws s3 ls {input_s3_url}

In [ ]:
# You can run your Python code locally and verify the corectness before constructing a pipeline
r_preprocess = preprocess(
    input_data_s3_path=input_s3_url,
    output_s3_prefix=output_s3_prefix,
    tracking_server_arn=mlflow_arn,
    experiment_name=f"local-test-{current_timestamp}"
)
r_preprocess

In [ ]:
# check that the function generated output
!aws s3 ls {output_s3_prefix}/test/

#### 훈련 단계
첫째, SageMaker 내장 알고리즘 훈련 작업으로 모델 훈련을 원격으로 실행합니다.

둘째, 훈련된 모델을 사용하여 평가 스크립트를 로컬 Python 함수로 테스트합니다.

셋째, 이 Python 함수를 사용하여 파이프라인에서 평가 단계를 구성합니다.

In [ ]:
# use sagemaker.Session() in the estimator to run a training job immediately
estimator = get_xgb_estimator(
    session=sagemaker.Session(),
    instance_type=train_instance_type,
    output_s3_url=output_s3_url,
    base_job_name=f"{project}-train",
)

In [ ]:
# Set up the training inputs using the outputs from preprocess function
training_inputs = {
    "train": TrainingInput(
        s3_data=r_preprocess['train_data'],
        content_type="text/csv",
    ),
    "validation": TrainingInput(
        s3_data=r_preprocess['validation_data'],
        content_type="text/csv",
    ),
}

다음 코드 셀은 Estimator를 훈련시킵니다. 훈련 작업이 완료될 때까지 3분 미만 기다립니다. 코드는 또한 훈련된 모델을 MLflow 실험 실행에 기록한다는 점에 유의하세요.

In [ ]:
from pipeline_steps.evaluate import load_model

# Run the training job
mlflow.set_experiment(r_preprocess['experiment_name'])
with mlflow.start_run(
    run_name=f"training-{strftime('%d-%H-%M-%S', gmtime())}",
    description="training in the notebook 03 with a training job") as run:
    mlflow.log_params(estimator.hyperparameters())
    
    estimator.fit(training_inputs)

    mlflow.log_param("training job name", estimator.latest_training_job.name)
    mlflow.log_metrics({i['metric_name'].replace(':', '_'):i['value'] for i in estimator.training_job_analytics.dataframe().iloc})
    mlflow.xgboost.log_model(load_model(estimator.model_data), artifact_path="xgboost")

#### 평가 단계
모델 성능이 지정된 임계값을 충족하는지 확인하는 모델 평가 스크립트를 로컬 Python 함수로 생성합니다. Python 코드는 `./pipeline_steps` 폴더의 `evaluate.py` 파일에 있습니다.

In [ ]:
from pipeline_steps.evaluate import evaluate

In [ ]:
# !pygmentize  pipeline_steps/evaluation.py

In [ ]:
# check that the training job you've just run created a model file
!aws s3 ls {estimator.model_data}

이제 평가 스크립트에서 훈련된 모델을 로드하고 스크립트가 올바르게 실행되는지 확인합니다. `preprocess` 함수의 출력 매개변수와 훈련 작업의 모델 아티팩트 사용법에 주목하세요.

In [ ]:
# Run the evaluation code locally
r_eval = evaluate(
    test_x_data_s3_path=r_preprocess['test_x_data'],
    test_y_data_s3_path=r_preprocess['test_y_data'],
    model_s3_path=estimator.model_data,
    output_s3_prefix=output_s3_prefix,
    tracking_server_arn=mlflow_arn,
    experiment_name=r_preprocess['experiment_name'],
)
r_eval

In [ ]:
# check that the evaluation function generated output
!aws s3 ls {output_s3_prefix}/prediction_baseline/

#### 모델 등록 단계
등록 단계는 SageMaker 모델을 생성하고 [모델 패키지 그룹](https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-model-group.html) 내의 SageMaker 모델 레지스트리에 모델의 새 버전을 등록합니다. 

이 단계도 로컬 Python 함수로 구현합니다. 코드는 `./pipeline_steps` 폴더의 `register.py` 파일에 제공됩니다.

In [ ]:
from pipeline_steps.register import register

In [ ]:
# !pygmentize  pipeline_steps/register.py

In [ ]:
# Run the model register code locally
r_register = register(
    training_job_name=estimator.latest_training_job.name,
    model_package_group_name=model_package_group_name,
    model_approval_status=model_approval_status,
    evaluation_result=r_eval['evaluation_result'],
    output_s3_prefix=output_s3_url,
    tracking_server_arn=mlflow_arn,
    experiment_name=r_preprocess['experiment_name'],
)
r_register

In [ ]:
# check that a new model version has been registered in the model package group
boto3.client('sagemaker').describe_model_package(ModelPackageName=r_register['model_package_arn'])

### 파이프라인 구성
로컬 테스트 후에는 변경 없이 동일한 Python 코드를 사용하여 파이프라인을 구성할 수 있습니다.

다음 셀은 이전에 개발하고 테스트한 단계로 파이프라인을 생성합니다. 동일한 파이프라인에서 `@step` 데코레이터가 적용된 함수(preprocess, evaluate, register)와 기존 파이프라인 단계(train)를 SageMaker 작업으로 혼합하여 사용하고 이들 간에 데이터를 전달한다는 점에 주목하세요.

SageMaker가 파이프라인 단계 간의 데이터 종속성을 기반으로 처리 플로우를 자동으로 도출하므로 단계의 순서를 수동으로 정의할 필요가 없습니다. 또한 SageMaker가 데이터 플로우를 자동으로 처리하므로 한 파이프라인 단계에서 다른 단계로 아티팩트와 데이터셋의 전송을 관리할 필요가 없습니다.

훈련 단계의 추정기 객체에서 Session 대신 [`PipelineSession`](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.pipeline_context.PipelineSession) 사용에 주목하세요. 파이프라인을 구성할 때는 SageMaker `Session`처럼 즉시 실행하지 않고 파이프라인 실행 시점에 작업을 시작하기 위해 `Estimator` 또는 `Processor`의 생성자에 PipelineSession 객체를 제공해야 합니다.

In [ ]:
# preprocess data step
step_preprocess = step(
    preprocess, 
    instance_type=process_instance_type_param,
    name=f"{project}-preprocess",
    keep_alive_period_in_seconds=3600,
)(
    input_data_s3_path=input_s3_url_param,
    output_s3_prefix=output_s3_prefix,
    tracking_server_arn=tracking_server_arn_param,
    experiment_name=experiment_name,
    pipeline_run_name=ExecutionVariables.PIPELINE_EXECUTION_ID,
)

cache_config = CacheConfig(enable_caching=True)
cache_config.expire_after = "p30d"

# train step
step_train = TrainingStep(
    name=f"{project}-train",
    step_args=get_xgb_estimator(
        session=PipelineSession(),
        instance_type=train_instance_type_param,
        output_s3_url=output_s3_url,
        base_job_name=f"{project}-train",
    ).fit(
        {
            "train": TrainingInput(
                step_preprocess['train_data'],
                content_type="text/csv",
            ),
            "validation": TrainingInput(
                step_preprocess['validation_data'],
                content_type="text/csv",
            ),
        }
    ),
    cache_config=cache_config,
)    

# evaluate step
step_evaluate = step(
    evaluate,
    instance_type=process_instance_type_param,
    name=f"{project}-evaluate",
    keep_alive_period_in_seconds=3600,
)(
    test_x_data_s3_path=step_preprocess['test_x_data'],
    test_y_data_s3_path=step_preprocess['test_y_data'],
    model_s3_path=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    output_s3_prefix=output_s3_prefix,
    tracking_server_arn=tracking_server_arn_param,
    experiment_name=step_preprocess['experiment_name'],
    pipeline_run_id=step_preprocess['pipeline_run_id'],
)

# register model step
step_register = step(
        register,
        name=f"{project}-register",
        keep_alive_period_in_seconds=3600,
    )(
        training_job_name=step_train.properties.TrainingJobName,
        model_package_group_name=model_package_group_name_param,
        model_approval_status=model_approval_status_param,
        evaluation_result=step_evaluate['evaluation_result'],
        output_s3_prefix=output_s3_url,
        tracking_server_arn=tracking_server_arn_param,
        experiment_name=step_preprocess['experiment_name'],
        pipeline_run_id=step_preprocess['pipeline_run_id'],
    )

# fail the pipeline execution step
step_fail = FailStep(
    name=f"{project}-fail",
    error_message=Join(on=" ", values=["Execution failed due to AUC Score < ", test_score_threshold_param]),
)

# condition to check in the condition step
condition_gte = ConditionGreaterThanOrEqualTo(
        left=step_evaluate['evaluation_result']['classification_metrics']['auc_score']['value'],  
        right=test_score_threshold_param,
)

# conditional register step
step_conditional_register = ConditionStep(
    name=f"{project}-check-metrics",
    conditions=[condition_gte],
    if_steps=[step_register],
    else_steps=[step_fail],
)

# Create a pipeline object
pipeline = Pipeline(
    name=f"{pipeline_name}",
    parameters=[
        input_s3_url_param,
        process_instance_type_param,
        train_instance_type_param,
        model_approval_status_param,
        test_score_threshold_param,
        model_package_group_name_param,
        tracking_server_arn_param,
    ],
    steps=[step_conditional_register],
    pipeline_definition_config=PipelineDefinitionConfig(use_custom_job_prefix=True)
)

파이프라인에 두 개의 단계를 더 추가했습니다: 실패 단계와 조건 단계입니다.

#### 실패 단계
Pipelines [FailStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.fail_step.FailStep)은 모델 성능 메트릭이 지정된 임계값을 충족하지 않으면 파이프라인 실행을 중지합니다.

#### 조건 단계
조건 단계는 평가 단계에서 계산된 모델 성능 점수를 확인하고 조건부로 모델을 생성하여 모델 레지스트리에 등록하거나 파이프라인 실행을 중지하고 실패시킵니다.

#### 파이프라인 Upsert
이제 파이프라인을 생성합니다. 동일한 이름의 파이프라인이 이미 존재하면 SageMaker가 업데이트합니다.

`Pipeline` 생성자에는 마지막 단계만 전달하면 됩니다. SDK는 단계 간의 데이터 종속성을 기반으로 파이프라인 DAG를 자동으로 구축합니다. 자세한 내용은 [개발자 가이드](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-step-decorator-create-pipeline.html#pipelines-step-define-delayed)를 참조하세요.

In [ ]:
# Upsert operation serialize the function code, arguments, and other artefacts to S3 where it can be accessed during pipeline's runtime
pipeline.upsert(role_arn=sm_role)

Studio UI에서 생성된 파이프라인을 보려면 아래 코드 셀에서 생성된 링크를 클릭하세요:

In [ ]:
# Show the pipeline link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/pipelines/{}/graph">the pipeline</a> in the Studio UI</b>'.format(
            domain_id, region, pipeline_name))
)

파이프라인 단계 간의 데이터 종속성을 기반으로 SageMaker는 파이프라인의 데이터 플로우와 함께 다음 DAG를 구축합니다:

![](img/pipeline-graph.png)

### 파이프라인 실행
첫 번째 파이프라인 실행은 약 17-20분이 걸립니다. 웜 풀 재사용을 위한 단계 정의의 `keep_alive_period_in_seconds` 매개변수와 단계 결과 캐싱을 위한 훈련 단계의 `CacheConfig` 사용에 주목하세요.
캐싱과 웜 풀 사용으로 인해 후속 파이프라인 실행은 약 7분이 걸립니다.

In [ ]:
pipeline_execution = pipeline.start()
pipeline_execution.describe()

In [ ]:
# Uncomment if you would like to wait in the notebook until this execution completes
# pipeline_execution.wait() 
pipeline_execution.list_steps()

다음 코드 셀에서 생성된 링크를 클릭하여 Studio UI에서 파이프라인 실행을 확인할 수 있습니다:

In [ ]:
# Show the pipeline execution link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/pipelines/{}/executions/{}/graph">the pipeline execution</a> in the Studio UI</b>'.format(
            domain_id, region, pipeline_name, pipeline_execution.describe()['PipelineExecutionArn'].split('/')[-1]))
)

Studio UI에서 파이프라인을 관리하려면 왼쪽 탐색 메뉴에서 **Pipelines**를 선택한 다음 특정 파이프라인을 선택하여 파이프라인의 실행과 모든 세부 정보를 확인합니다:

![](img/pipelines-pane.png)

각 실행에 대해 실행 그래프를 열고 각 파이프라인 단계의 세부 정보를 볼 수 있습니다:

![](img/pipeline-execution-graph.png)

MLflow UX에서 파이프라인 실행, 아티팩트, 데이터셋 및 모델을 추적할 수 있습니다:

![](img/mlflow-pipeline-executions.png)

### 파이프라인 정의 이해

In [ ]:
pipeline_definition = json.loads(pipeline.describe()['PipelineDefinition'])
pipeline_definition

파이프라인 정의 JSON을 살펴보고 이해해보세요. 예를 들어, 파이프라인 매개변수가 어떻게 정의되고 사용되는지 확인할 수 있습니다.

정의:
```json
'Parameters': [{'Name': 'ProcessingInstanceType',
  'Type': 'String',
  'DefaultValue': 'ml.m5.large'},
              ...
              ]
```

매개변수 치환:
```json
'Arguments': {'ProcessingResources': {'ClusterConfig': {'InstanceType':{'Get':'Parameters.ProcessingInstanceType'}
```

이제 `preprocess` 단계의 정의를 찾아보세요:

```json
{'Name': 'from-idea-to-prod-preprocess',
  'Type': 'Training',
  'Arguments': {'TrainingJobName': 'preprocess',
   'RoleArn': 'arn:aws:iam::906545278380:role/service-role/AmazonSageMaker-ExecutionRole-20240214T222844',
   'StoppingCondition': {'MaxRuntimeInSeconds': 86400},
   'RetryStrategy': {'MaximumRetryAttempts': 1},
   'InputDataConfig': [{'ChannelName': 'sagemaker_remote_function_bootstrap',
     'DataSource': {'S3DataSource': {'S3Uri': 's3://sagemaker-us-east-1-906545278380/from-idea-to-prod-pipeline-03-09-21-09/sagemaker_remote_function_bootstrap',
       'S3DataType': 'S3Prefix'}}},
```

SageMaker가 기본적으로 이 JupyterLab 노트북에 사용된 커널 이미지와 동일한 이미지를 가진 컨테이너에서 Python 스크립트를 실행하는 것을 볼 수 있습니다:

```json
'AlgorithmSpecification': {'TrainingImage': '885854791233.dkr.ecr.us-east-1.amazonaws.com/sagemaker-distribution-prod@sha256:7c07530831d3d25b27a77b6a77f9801eec01b7b80c69ca1aa2c9eae3df00887d',
```

파이프라인을 수동으로 생성하려면 JSON으로 작업하고 [SageMaker Pipeline Definition JSON Schema](https://aws-sagemaker-mlops.github.io/sagemaker-model-building-pipeline-definition-JSON-schema/index.html)를 따를 수 있습니다.

## 파이프라인에 배치 변환 및 품질 검사 추가
모델 구축 파이프라인에 추가 단계를 통합하여 필요한 모든 작업을 자동화할 수 있습니다. 이 섹션에서는 다음 단계를 추가합니다:
- [`QualityCheckStep`](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-quality-check)을 사용한 데이터와 모델 모두에 대한 품질 검사 및 기준선 계산
- [`TransformStep`](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-transform)을 사용한 배치 변환

모델 품질 검사에 대한 더 자세한 예제는 예제 노트북 [SageMaker Pipelines integration with Model Monitor and Clarify](https://github.com/aws/amazon-sagemaker-examples/blob/main/sagemaker-pipelines/tabular/model-monitor-clarify-pipelines/sagemaker-pipeline-model-monitor-clarify-steps.ipynb)를 참조하세요.

데이터 및 모델 품질 생명주기를 이해하려면 개발자 가이드 [Amazon SageMaker Model Building Pipelines의 ClarifyCheck 및 QualityCheck 단계를 통한 기준선 계산, 드리프트 감지 및 생명주기](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-quality-clarify-baseline-lifecycle.html)를 참조하세요.

In [ ]:
from sagemaker.transformer import Transformer
from sagemaker.inputs import TransformInput
from sagemaker.workflow.steps import TransformStep
from sagemaker.inputs import CreateModelInput

### 품질 검사
파이프라인을 위한 데이터 및 모델 품질 검사 단계의 정의부터 시작합니다. 데이터 및 모델 품질 검사 단계는 파이프라인의 `preprocess` 및 `evaluate` 단계의 데이터를 사용합니다.

In [ ]:
# Parameters to control data quality check
skip_check_data_quality_param = ParameterBoolean(name="SkipDataQualityCheck", default_value=True)
register_new_baseline_data_quality_param = ParameterBoolean(
    name="RegisterNewDataQualityBaseline", default_value=True
)

# Parameters to control model quality check
skip_check_model_quality_param = ParameterBoolean(name="SkipModelQualityCheck", default_value=True)
register_new_baseline_model_quality_param = ParameterBoolean(
    name="RegisterNewModelQualityBaseline", default_value=True
)

# Job configuration for both data and model quality check steps
check_job_config = CheckJobConfig(
    role=sm_role,
    instance_count=1,
    instance_type=process_instance_type_param,
)

# Configuration for data quality check step
data_quality_check_config = DataQualityCheckConfig(
    baseline_dataset=step_preprocess['baseline_data'],
    dataset_format=DatasetFormat.csv(header=False),
    output_s3_uri=baseline_results_s3_url,
)

# Configuration for model quality check step
model_quality_check_config = ModelQualityCheckConfig(
    baseline_dataset=step_evaluate['prediction_baseline_data'],
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=prediction_baseline_results_s3_url,
    problem_type="BinaryClassification",
    inference_attribute= "prediction", # The column in the dataset that contains predictions
    probability_attribute= "probability", # The column in the dataset that contains probabilities
    ground_truth_attribute= "label", # The column in the dataset that contains ground truth labels
)

cache_config = CacheConfig(enable_caching=True)
cache_config.expire_after = "p30d"

# Data quality check step
step_data_quality_check = QualityCheckStep(
    name=f"{project}-data-quality",
    quality_check_config=data_quality_check_config,
    check_job_config=check_job_config,
    skip_check=skip_check_data_quality_param,
    register_new_baseline=register_new_baseline_data_quality_param,
    model_package_group_name=model_package_group_name_param,
    cache_config=cache_config,
)

# Model quality check step
step_model_quality_check = QualityCheckStep(
    name=f"{project}-model-quality",
    quality_check_config=model_quality_check_config,
    check_job_config=check_job_config,
    skip_check=skip_check_model_quality_param,
    register_new_baseline=register_new_baseline_model_quality_param,
    model_package_group_name=model_package_group_name_param,
    cache_config=cache_config,
)

### 배치 변환
파이프라인에 변환 단계를 추가합니다.

In [ ]:
session = PipelineSession()

# create model step
step_create_model = ModelStep(
    name=f"{project}-model",
    step_args=Model(
        image_uri=XGBOOST_IMAGE_URI,        
        model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
        name=f"from-idea-to-prod-xgboost-model",
        sagemaker_session=session,
        role=sm_role,
    ).create(instance_type="ml.m5.large"),
)

# create the transform step
step_transform = TransformStep(
    name=f"{project}-transform", 
    step_args=Transformer(
        model_name=step_create_model.properties.ModelName,
        instance_type=train_instance_type_param,
        instance_count=1,
        accept="text/csv",
        assemble_with="Line",
        output_path=f"{output_s3_prefix}/transform",
        sagemaker_session=session,
        base_transform_job_name=f"{project}-transform",
    ).transform(    
        data=step_preprocess["test_x_data"],
        content_type="text/csv",
        split_type="Line", 
    ),
    cache_config=cache_config,
)

생성된 모델 및 데이터 품질 베이스라인을 모델 등록 단계에 포함해야 합니다.

In [ ]:
# re-define the register model step to use the calculated model and data quality baselines
step_register = step(
        register,
        name=f"{project}-register",
        keep_alive_period_in_seconds=3600,
    )(
        training_job_name=step_train.properties.TrainingJobName,
        model_package_group_name=model_package_group_name_param,
        model_approval_status=model_approval_status_param,
        evaluation_result=step_evaluate['evaluation_result'],
        output_s3_prefix=output_s3_url,
        tracking_server_arn=tracking_server_arn_param,
        model_statistics_s3_path=step_model_quality_check.properties.CalculatedBaselineStatistics,
        model_constraints_s3_path=step_model_quality_check.properties.CalculatedBaselineConstraints,
        model_data_statistics_s3_path=step_data_quality_check.properties.CalculatedBaselineStatistics,
        model_data_constraints_s3_path=step_data_quality_check.properties.CalculatedBaselineConstraints,
        experiment_name=step_preprocess['experiment_name'],
        pipeline_run_id=step_preprocess['pipeline_run_id'],
    )

# re-define the conditional register step with the new step_register
step_conditional_register = ConditionStep(
    name=f"{project}-check-metrics",
    conditions=[condition_gte],
    if_steps=[step_register, step_transform],
    else_steps=[step_fail],
)

In [ ]:
# create a pipeline object
pipeline = Pipeline(
    name=f"{pipeline_name}",
    parameters=[
        input_s3_url_param,
        process_instance_type_param,
        train_instance_type_param,
        model_approval_status_param,
        test_score_threshold_param,
        model_package_group_name_param,
        tracking_server_arn_param,
        skip_check_data_quality_param,
        skip_check_model_quality_param,
        register_new_baseline_data_quality_param,
        register_new_baseline_model_quality_param,
    ],
    steps=[step_conditional_register],
    pipeline_definition_config=PipelineDefinitionConfig(use_custom_job_prefix=True)
)

In [ ]:
# update the pipeline
pipeline.upsert(role_arn=sm_role, parallelism_config=ParallelismConfiguration(5).to_request())

In [ ]:
# Show the pipeline link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/pipelines/{}/graph">the pipeline</a> in the Studio UI</b>'.format(
            domain_id, region, pipeline_name))
)

새 파이프라인에는 이제 데이터 및 모델 품질 검사와 배치 변환을 위한 단계들이 포함되어 있습니다:

![](img/pipeline-graph-with-transform.png)

### 새 파이프라인 실행
파이프라인을 처음 실행할 때는 `skip_check_...` 및 `register_new_baseline_...` 매개변수를 기본값 `(True, True)`로 설정해야 품질 검사가 건너뛰어지고 새로 계산된 베이스라인이 모델 버전에 등록됩니다.

In [ ]:
pipeline_execution = pipeline.start()
pipeline_execution.describe()

In [ ]:
# Uncomment if you would like to wait in the notebook until this execution completes
# pipeline_execution.wait() 
pipeline_execution.list_steps()

파이프라인을 한 번 더 실행하려면 `SkipDataQualityCheck` 매개변수를 `True`로 설정하여 생성된 베이스라인을 입력 데이터셋 및 모델 출력과 비교하는 데이터 품질 검사를 실행할 수 있습니다.

In [ ]:
# Uncomment if you would like to start one more execution
# pipeline_execution = pipeline.start(
#     parameters=dict(
#         SkipDataQualityCheck=False,
#     )
# )

### SageMaker Pipelines의 추가 기능
SageMaker Pipelines의 더 유용한 기능들을 직접 탐색해보세요. 예를 들어 [파이프라인 단계의 선택적 실행](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-selective-ex.html), [교차 계정 지원](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-xaccount.html), [예약된 파이프라인 실행](https://docs.aws.amazon.com/sagemaker/latest/dg/pipeline-eventbridge.html), 또는 [로컬 모드](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-local-mode.html) 등이 있습니다.

파이프라인에 더 많은 모니터링 기능을 추가해야 하는 경우, [MonitorBatchTransformStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.monitor_batch_transform_step.MonitorBatchTransformStep)을 사용하여 변환 단계를 품질 검사, 편향 탐지, 설명 가능성 보고서와 결합할 수 있습니다.

---

## 선택사항: 피처 스토어 추가
이 섹션에서는 SageMaker Feature Store를 사용하여 모델 훈련을 위한 피처와 데이터셋을 관리합니다.

<div class="alert alert-info"> 이 섹션은 선택사항이며 워크샵 과정에 필수가 아닙니다. 여기서 멈추고 다음 노트북으로 이동할 수 있습니다.</div>

In [ ]:
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.feature_store.inputs import TableFormatEnum
from sagemaker.feature_store.feature_processor import CSVDataSource, feature_processor, to_pipeline
from sagemaker.remote_function import remote
from sagemaker.workflow.function_step import step
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timezone, date
from time import gmtime, strftime, sleep
import time

In [ ]:
%store -r 

In [ ]:
session = sagemaker.Session()
project = "from-idea-to-prod"
current_timestamp = strftime('%d-%H-%M-%S', gmtime())

In [ ]:
feature_store_bucket_prefix = 'from-idea-to-prod/feature-store'
%store feature_store_bucket_prefix

### 원시 데이터를 훈련 준비된 피처로 변환
먼저 데이터셋에서 스키마를 추출할 수 있도록 원시 데이터를 피처로 변환합니다. 피처 그룹 정의를 위해 데이터 스키마가 필요합니다.

In [ ]:
# Load original raw data
df_data = pd.read_csv(dataset_file_local_path, sep=";")
pd.set_option("display.max_columns", 500)
df_data

원시 데이터에 피처 엔지니어링을 적용합니다:

In [ ]:
target_col = "y"

# Indicator variable to capture when pdays takes a value of 999
df_data["no_previous_contact"] = np.where(df_data["pdays"] == 999, 1, 0)

# Indicator for individuals not actively employed
df_data["not_working"] = np.where(
    np.in1d(df_data["job"], ["student", "retired", "unemployed"]), 1, 0
)

# remove unnecessary data
df_model_data = df_data.drop(
    ["duration", "emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed"],
    axis=1,
)

bins = [18, 30, 40, 50, 60, 70, 90]
labels = ['18-29', '30-39', '40-49', '50-59', '60-69', '70-plus']

df_model_data['age_range'] = pd.cut(df_model_data.age, bins, labels=labels, include_lowest=True)
df_model_data = pd.concat([df_model_data, pd.get_dummies(df_model_data['age_range'], prefix='age', dtype=int)], axis=1)
df_model_data.drop('age', axis=1, inplace=True)
df_model_data.drop('age_range', axis=1, inplace=True)

scaled_features = ['pdays', 'previous', 'campaign']
df_model_data[scaled_features] = MinMaxScaler().fit_transform(df_model_data[scaled_features])

df_model_data = pd.get_dummies(df_model_data, dtype=int)  # Convert categorical variables to sets of indicators

# Replace "y_no" and "y_yes" with a single label column, and bring it to the front:
df_model_data = pd.concat(
    [
        df_model_data["y_yes"].rename(target_col),
        df_model_data.drop(["y_no", "y_yes"], axis=1),
    ],
    axis=1,
)

In [ ]:
def generate_event_timestamp():
    # naive datetime representing local time
    naive_dt = datetime.now()
    # take timezone into account
    aware_dt = naive_dt.astimezone()
    # time in UTC
    utc_dt = aware_dt.astimezone(timezone.utc)
    # transform to ISO-8601 format
    event_time = utc_dt.isoformat(timespec='milliseconds')
    event_time = event_time.replace('+00:00', 'Z')
    return event_time

<div style="border: 4px solid coral; text-align: center; margin: auto;">
각 피처 그룹에 필요한 두 필드이므로 데이터셋에 `event_time` 및 `record_id` 열을 추가하세요:
</div>

In [ ]:
df_model_data['event_time'] = generate_event_timestamp()
df_model_data['record_id'] = [f'R{i}' for i in range(len(df_model_data))]

피처 이름에는 '.'이 포함될 수 없고 '_'로 끝날 수 없습니다. 또한 열 이름을 변환할 때 열 이름에서 '-'를 제거하세요:

In [ ]:
def convert_col_name(c):
    return c.replace('.', '_').replace('-', '_').rstrip('_')

In [ ]:
df_model_data = df_model_data.rename(columns=convert_col_name)
df_model_data = df_model_data.convert_dtypes(infer_objects=True, convert_boolean=False)
df_model_data['record_id'] = df_model_data['record_id'].astype('string')
df_model_data['event_time'] = df_model_data['event_time'].astype('string')

In [ ]:
df_model_data.dtypes

In [ ]:
df_model_data.columns

In [ ]:
df_model_data.head()

In [ ]:
df_model_data.shape

In [ ]:
df_model_data.to_csv('./data/feature_dataset.csv', index=False)

In [ ]:
record_count = len(df_model_data)

In [ ]:
record_count

### 피처 그룹 생성
이제 데이터셋 스키마로 피처 그룹을 생성할 모든 준비가 완료되었습니다.

In [ ]:
dataset_feature_group_name = f'{project}-{current_timestamp}'

In [ ]:
%store dataset_feature_group_name

In [ ]:
dataset_feature_group = FeatureGroup(name=dataset_feature_group_name, sagemaker_session=session)

In [ ]:
# use the DataFrame to extract the feature group definitions
dataset_feature_group.load_feature_definitions(data_frame=df_model_data)

In [ ]:
def wait_for_feature_group_creation_complete(feature_group):
    status = feature_group.describe().get('FeatureGroupStatus')
    print(f'Initial status: {status}')
    while status == 'Creating':
        print(f'Waiting for feature group: {feature_group.name} to be created ...')
        time.sleep(5)
        status = feature_group.describe().get('FeatureGroupStatus')
    if status != 'Created':
        raise SystemExit(f'Failed to create feature group {feature_group.name}: {status}')
    print(f'FeatureGroup {feature_group.name} was successfully created.')

In [ ]:
dataset_feature_group.create(
    s3_uri=f's3://{bucket_name}/{feature_store_bucket_prefix}', 
    record_identifier_name='record_id', 
    event_time_feature_name='event_time', 
    role_arn=sm_role, 
    enable_online_store=False,
    table_format=TableFormatEnum.ICEBERG 
)

<div style="border: 4px solid coral; text-align: center; margin: auto;">
피처 그룹이 생성되고 사용할 준비가 될 때까지 기다리세요. 1분 미만이 걸립니다.
</div>

In [ ]:
wait_for_feature_group_creation_complete(dataset_feature_group)

자체 계정이나 AWS 주도 이벤트를 통해 프로비저닝되지 않은 계정에서 이 워크샵을 실행하는 경우 권한 설정으로 인해 피처 그룹 생성에 문제가 있을 수 있습니다. 이 [피처 그룹 생성 문제 해결 가이드](https://repost.aws/knowledge-center/sagemaker-featuregroup-troubleshooting)를 따르세요. 권한 문제는 대부분 SageMaker 실행 역할에 대한 LakeFormation 권한 누락과 관련이 있습니다. SageMaker 실행 역할에 데이터베이스 `sagemaker_featurestore`에 대한 액세스 권한을 부여해야 합니다.

In [ ]:
dataset_feature_group.describe()

피처 그룹이 사용할 준비가 되었습니다. 이제 데이터를 수집해야 합니다.

### SageMaker 파이프라인을 통해 피처 그룹에 데이터 수집
이전 섹션과 마찬가지로 [`@step`](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-step-decorator-create-pipeline.html) 데코레이터를 사용하여 피처 수집 파이프라인을 생성합니다.

이전의 모든 피처 변환 및 수집 코드를 원격 함수로 컴파일합니다. 함수 코드는 `.\pipeline_steps` 폴더의 `ingest.py` 파일에 있습니다. Python SDK [`FeatureGroup.ingest()`](https://sagemaker.readthedocs.io/en/stable/api/prep_data/feature_store.html#sagemaker.feature_store.feature_group.FeatureGroup.ingest) 메서드를 사용하여 pandas DataFrame의 내용을 피처 그룹에 수집합니다.

In [ ]:
# Python function code is in the local files
from pipeline_steps.ingest import process_and_ingest

먼저 피처 스토어 수집을 로컬에서 실행합니다:

In [ ]:
process_and_ingest(input_s3_url, dataset_feature_group.describe()['FeatureGroupArn'])

수집 파이프라인을 정의합니다:

In [ ]:
# Create parameters for the feature store ingestion pipeline
input_s3_url_param = ParameterString(
    name="InputDataUrl",
    default_value=input_s3_url,
)

feature_group_name_param = ParameterString(
    name="FeatureGroupName",
    default_value=dataset_feature_group.describe()['FeatureGroupArn'],
)

In [ ]:
# feature store ingest step
fs_ingest = step(
    process_and_ingest, 
    name=f'{project}-fs-ingest',
)(
    input_s3_url=input_s3_url_param,
    feature_group_name=feature_group_name_param,
)

# create a pipeline with an ingest step
pipeline_fs_ingest = Pipeline(
    name=f"{pipeline_name}-fs-ingest",
    parameters=[
        input_s3_url_param,
        feature_group_name_param
    ],
    steps=[fs_ingest]
)

In [ ]:
pipeline_fs_ingest.upsert(role_arn=sm_role)

이 피처 수집 파이프라인은 단 하나의 단계로만 구성됩니다. 다음 셀에서 생성된 링크를 따라가면 Studio UX에서 파이프라인을 확인할 수 있습니다.

In [ ]:
# Show the pipeline link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/pipelines/{}/graph">the pipeline</a> in the Studio UI</b>'.format(
            domain_id, region, pipeline_fs_ingest.name))
)

In [ ]:
execution_fs_ingest = pipeline_fs_ingest.start()

In [ ]:
execution_fs_ingest.describe()

<div class="alert alert-info">파이프라인이 완료되고 피처가 피처 그룹에 수집될 때까지 기다려야 합니다. 실행에는 약 5분이 걸립니다.</div>

In [ ]:
# You need to wait until the feature ingestion pipeline execution completes
execution_fs_ingest.wait()
execution_fs_ingest.list_steps()

In [ ]:
assert execution_fs_ingest.list_steps()[0]['StepStatus'] == 'Succeeded', 'Ingestion pipepline execution status must be Succeeded!'

### Studio UI의 Feature Store
Studio UI에서 피처 스토어와 피처 그룹을 탐색할 수 있습니다. **Data** > **Feature Store**로 이동하세요:

![](img/feature-store-studio-ui.png)

UI에서 피처, 피처 그룹 메타데이터를 탐색하고, 샘플 쿼리 및 관련 파이프라인 실행을 확인할 수 있습니다.

### 피처 그룹에서 수집된 피처 검색

오프라인 피처 스토어에서 피처를 추출하는 방법은 여러 가지가 있습니다. 예를 들어, [Amazon Athena 쿼리](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store-create-a-dataset.html#feature-store-athena-sample-queries)를 사용하여 오프라인 스토어에 저장된 데이터를 쿼리하고 조인하거나, [Offline Store Python SDK](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store-create-a-dataset.html#feature-store-dataset-python-sdk)를 사용할 수 있습니다. Python SDK를 사용하여 피처를 추출하고 모델 구축 파이프라인을 위한 데이터셋을 생성합니다.

<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <p style=" text-align: center; margin: auto;">오프라인 스토어로의 수집은 버퍼링되며 데이터가 피처 그룹에 나타나는 데 최대 15분이 걸립니다. 피처가 수집되고 오프라인 스토어에서 사용 가능해진 후에는 이를 쿼리하고 모델 훈련 및 스코어링을 위한 데이터셋을 생성할 수 있습니다.
    </p>
</div>

In [ ]:
sagemaker_client = boto3.client('sagemaker')
output_location = f's3://{bucket_name}/{feature_store_bucket_prefix}/offline-store/query_results/'

In [ ]:
def get_historical_record_count(fg):
    fs_query = dataset_feature_group.athena_query()
    query_string = f'SELECT COUNT(*) FROM "' + fs_query.table_name + f'"'
    output_location =  f's3://{bucket_name}/{feature_store_bucket_prefix}/offline-store/query_results/'

    fs_query.run(query_string=query_string, output_location=output_location)
    fs_query.wait()
    fs_df = fs_query.as_dataframe()
    
    return fs_df.iat[0, 0]

<div class="alert alert-info">다음 코드 셀은 피처가 오프라인 스토어에 나타날 때까지 기다립니다. 최대 15분이 걸릴 수 있습니다. 이전에 이미 피처를 수집했다면 첫 번째 쿼리 후에 셀이 종료됩니다.</div>

In [ ]:
# Before accessing the feature data you need to check if the offline feature store was populated
offline_store_contents = None
while offline_store_contents is None:    
    fs_record_count = get_historical_record_count(dataset_feature_group)
    print(f"Total number of historical record in the {dataset_feature_group.name}: {fs_record_count}")

    if fs_record_count >= record_count:
        print(f'[{fs_record_count} feature records are available in offline store for {dataset_feature_group.name} feature group]')
        offline_store_contents = fs_record_count
    else:
        print('[Waiting for data arrives in offline store ...]')
        time.sleep(60)

#### Amazon SageMaker Python SDK(DatasetBuilder)를 사용하여 피처 스토어 쿼리
이 섹션에서는 [`DatasetBuilder`](https://sagemaker.readthedocs.io/en/stable/api/prep_data/feature_store.html#sagemaker.feature_store.dataset_builder.DatasetBuilder)를 사용하여 피처 그룹에서 데이터를 가져오는 방법을 보여줍니다. 자세한 예제는 [개발자 가이드](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store-create-a-dataset.html)를 참조하세요.

In [ ]:
from sagemaker.feature_store.feature_store import FeatureStore

In [ ]:
region = boto3.Session().region_name
boto_session = boto3.Session(region_name=region)

s3_client = boto3.client('s3', region_name=region)
sagemaker_client = boto_session.client(service_name="sagemaker", region_name=region)
featurestore_runtime = boto_session.client(service_name="sagemaker-featurestore-runtime",region_name=region)

In [ ]:
# Create FeatureStore session object
feature_store_session = sagemaker.Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)

feature_store = FeatureStore(sagemaker_session=feature_store_session)

In [ ]:
included_feature_names = [f.feature_name for f in dataset_feature_group.feature_definitions]

In [ ]:
# Create dataset builder to retrieve the most recent version of each record
builder = feature_store.create_dataset(
    base=dataset_feature_group,
    # included_feature_names=included_feature_names,
    output_path=output_location,
).with_number_of_recent_records_by_record_identifier(1)

In [ ]:
df_dataset, query = builder.to_dataframe()

In [ ]:
df_dataset

### 모델 구축 파이프라인에 피처 그룹 통합
지금까지 모든 변환된 피처를 피처 스토어에 수집했습니다. 이 노트북의 마지막 단계로 S3 버킷에서 원시 데이터 파일을 로드하고 변환하는 대신 피처 그룹의 변환된 피처를 사용하도록 모델 구축 파이프라인을 적응시켜야 합니다.

피처 스토어에서 피처를 추출하고 훈련, 테스트, 검증 및 품질 기준선을 위한 데이터셋을 준비하는 코드는 `./pipeline_steps` 폴더의 Python 파일 `extract.py`에 있습니다.
모든 피처 엔지니어링이 피처 스토어에 피처를 수집하기 전에 수행되므로 스크립트에는 피처 처리 코드가 없다는 점에 유의하세요.

In [ ]:
# Python function code is in the local files
from pipeline_steps.extract import prepare_datasets

In [ ]:
# Run the function locally to verify everything works
r_extract = prepare_datasets(
    feature_group_name=dataset_feature_group_name,
    output_s3_prefix=output_s3_prefix,
    query_output_s3_path=output_location,
    tracking_server_arn=mlflow_arn,
    experiment_name=f"local-test-{current_timestamp}"
)
r_extract

방금 이 스크립트를 로컬에서 테스트했으니, 이제 모델 구축 파이프라인에 통합해보겠습니다. 다음 셀에는 파이프라인 구축을 위한 전체 스크립트가 포함되어 있습니다.

명확성을 위해 이 파이프라인에서는 배치 모니터링 및 품질 모니터링 단계가 생략되었습니다. **파이프라인에 배치 변환 및 품질 검사 추가** 섹션의 코드를 사용하여 이러한 단계를 자유롭게 추가하세요.

In [ ]:
from pipeline_steps.evaluate import evaluate
from pipeline_steps.register import register

In [ ]:
session = PipelineSession()
experiment_name = f"{project}-fs-pipeline-{current_timestamp}"
mlflow.set_experiment(experiment_name)

# extract features from the feature store step
fs_step_extract_featureset = step(
    prepare_datasets, 
    instance_type=process_instance_type_param,
    name=f"{project}-extract-featureset",
)(
    feature_group_name=feature_group_name_param,
    output_s3_prefix=output_s3_prefix,
    query_output_s3_path=output_location,
    tracking_server_arn=tracking_server_arn_param,
    experiment_name=experiment_name,
    pipeline_run_name=ExecutionVariables.PIPELINE_EXECUTION_ID,
)

cache_config = CacheConfig(enable_caching=True)
cache_config.expire_after = "p30d"

# train step
fs_step_train = TrainingStep(
    name=f"{project}-train",
    step_args=get_xgb_estimator(
        session=session,
        instance_type=train_instance_type_param,
        output_s3_url=output_s3_url,
        base_job_name=f"{project}-train",
    ).fit(
        {
            "train": TrainingInput(
                fs_step_extract_featureset['train_data'],
                content_type="text/csv",
            ),
            "validation": TrainingInput(
                fs_step_extract_featureset['validation_data'],
                content_type="text/csv",
            ),
        }
    ),
    cache_config=cache_config,
)    

# evaluate step
fs_step_evaluate = step(
    evaluate,
    instance_type=process_instance_type_param,
    name=f"{project}-evaluate",
)(
    test_x_data_s3_path=fs_step_extract_featureset['test_x_data'],
    test_y_data_s3_path=fs_step_extract_featureset['test_y_data'],
    model_s3_path=fs_step_train.properties.ModelArtifacts.S3ModelArtifacts,
    output_s3_prefix=output_s3_prefix,
    tracking_server_arn=tracking_server_arn_param,
    experiment_name=fs_step_extract_featureset['experiment_name'],
    pipeline_run_id=fs_step_extract_featureset['pipeline_run_id'],
)

# register model step
fs_step_register = step(
        register,
        name=f"{project}-register",
    )(
        training_job_name=fs_step_train.properties.TrainingJobName,
        model_package_group_name=model_package_group_name_param,
        model_approval_status=model_approval_status_param,
        evaluation_result=fs_step_evaluate['evaluation_result'],
        output_s3_prefix=output_s3_url,
        tracking_server_arn=tracking_server_arn_param,
        experiment_name=fs_step_extract_featureset['experiment_name'],
        pipeline_run_id=fs_step_extract_featureset['pipeline_run_id'],
    )

# fail the pipeline execution step
fs_step_fail = FailStep(
    name=f"{project}-fail",
    error_message=Join(on=" ", values=["Execution failed due to AUC Score < ", test_score_threshold_param]),
)

# condition to check in the condition step
fs_condition_gte = ConditionGreaterThanOrEqualTo(
        left=fs_step_evaluate['evaluation_result']['classification_metrics']['auc_score']['value'],  
        right=test_score_threshold_param,
)

# conditional register step
fs_step_conditional_register = ConditionStep(
    name=f"{project}-check-metrics",
    conditions=[fs_condition_gte],
    if_steps=[fs_step_register],
    else_steps=[fs_step_fail],
)

# Create a pipeline object
pipeline_feature_store = Pipeline(
    name=f"{pipeline_name}-fs",
    parameters=[
        feature_group_name_param,
        process_instance_type_param,
        train_instance_type_param,
        model_approval_status_param,
        test_score_threshold_param,
        model_package_group_name_param,
        tracking_server_arn_param,
    ],
    steps=[fs_step_conditional_register],
    pipeline_definition_config=PipelineDefinitionConfig(use_custom_job_prefix=True)
)

In [ ]:
pipeline_feature_store.upsert(role_arn=sm_role)

새 파이프라인은 첫 번째 파이프라인과 정확히 동일하게 보이지만 피처 엔지니어링 대신 피처셋 추출 단계인 `from-idea-to-prod-extract-featureset`를 가지고 있습니다. Studio UX에서 파이프라인을 보려면 다음 셀에서 생성된 링크를 클릭하세요.

In [ ]:
# Show the pipeline link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/pipelines/{}/graph">the pipeline</a> in the Studio UI</b>'.format(
            domain_id, region, pipeline_feature_store.name))
)

### 새 파이프라인 실행

In [ ]:
execution_feature_store = pipeline_feature_store.start()

In [ ]:
# Uncomment these two lines if you'd like to wait until execution is done
# execution_feature_store.wait()
# execution_feature_store.list_steps()

## 추가 읽기: 피처 변환 및 수집을 위한 Feature Processor 사용
SageMaker는 배치 데이터 소스에서 피처 그룹으로 데이터를 변환하고 수집할 수 있는 Spark 기반 [Feature Processor SDK](https://sagemaker.readthedocs.io/en/stable/api/prep_data/feature_store.html#feature-processor-decorator)를 제공합니다. [개발자 가이드](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store-feature-processing.html)의 설명과 예제를 읽어보세요.

피처 프로세서의 더 자세한 예제는 [feature store feature processor](https://github.com/aws/amazon-sagemaker-examples/blob/main/sagemaker-featurestore/feature_store_feature_processor.ipynb) 노트북을 참조하세요.

## 요약
이 노트북에서는 세 개의 SageMaker 파이프라인을 구축했습니다:
- 데이터 처리, 모델 훈련, 모델 평가 및 조건부 모델 등록 단계가 포함된 초기 모델 구축 파이프라인
- 데이터 및 모델 품질 모니터링과 배치 변환이 포함된 이 파이프라인의 다음 버전
- SageMaker Feature Store로의 피처 엔지니어링 및 수집 파이프라인
- 데이터 처리 단계가 피처셋 추출 단계로 대체된 초기 파이프라인의 적응 버전

---

## 워크샵 플로우 계속하기
이 실습을 완료한 후 4단계와 5단계 [노트북](04-sagemaker-project.ipynb)을 계속하거나 6단계 [노트북](06-monitoring.ipynb)으로 직접 이동할 수 있습니다:

- **4단계와 5단계 노트북**: SageMaker Projects를 사용하여 모델 구축 및 배포를 위한 CI/CD 자동화 파이프라인 구현

- **6단계 노트북**: 데이터 및 모델 품질 모니터링

## 실제 프로젝트를 위한 추가 개발 아이디어
- [편향 감지 및 모델 설명 가능성](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-clarify-check) 단계 추가. [SageMaker Clarify](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-configure-processing-jobs.html)에서 계산된 모델 메트릭을 모델 레지스트리의 모델 메타데이터에 추가
- 새 데이터셋이 Amazon S3 버킷에 업로드되는 즉시 ML 파이프라인의 이벤트 기반 실행 추가. [Amazon EventBridge 통합](https://docs.aws.amazon.com/sagemaker/latest/dg/pipeline-eventbridge.html#pipeline-eventbridge-schedule)을 사용하여 다양한 이벤트 기반 워크플로우 구현
- 파이프라인 실행을 위한 전용 IAM 실행 역할 사용
- S3 버킷 암호화 및 컨테이너 EBS 볼륨 암호화를 위한 AWS KMS 키를 사용하여 데이터 암호화 추가

## 추가 리소스
- [Automate Machine Learning Workflows](https://aws.amazon.com/getting-started/hands-on/machine-learning-tutorial-mlops-automate-ml-workflows/)
- [Amazon SageMaker Feature Store workshop](https://github.com/aws-samples/amazon-sagemaker-feature-store-end-to-end-workshop)
- [Amazon SageMaker Model Building Pipeline](https://github.com/aws/sagemaker-python-sdk/blob/master/doc/amazon_sagemaker_model_building_pipeline.rst)
- [MLOPs With SageMaker Pipelines Step Decorator](https://towardsaws.com/mlops-with-sagemaker-pipelines-step-decorator-bb63fce88846)

# kernel 정지

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>